In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector

I:\Anaconda\road\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
I:\Anaconda\road\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
I:\Anaconda\road\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
I:\Anaconda\road\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a syn

### 1 加载数据集

In [2]:
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)  # 加载数据，label用独热编码
max_step = 1001  # 迭代次数，每次是批量 不是跑完所有的数据
image_num = 3000  # 显示图片的数目
DIR = 'J:/tensorboard/'  # 文件路径
sess = tf.Session()
embedding = tf.Variable(tf.stack(mnist.test.images[:image_num]), trainable=False, name='embedding')   # 3000张图片stack成的变量


def variable_summary(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)  # 求平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)  # 标准差
        tf.summary.scalar('max', tf.reduce_max(var))  # 最大值
        tf.summary.scalar('min', tf.reduce_min(var))  # 最小值
        tf.summary.histogram('histogram', var)  # 直方图

W0813 01:18:15.634047  1932 deprecation.py:323] From <ipython-input-2-1d84208486bb>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0813 01:18:15.638051  1932 deprecation.py:323] From I:\Anaconda\road\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0813 01:18:15.641050  1932 deprecation.py:323] From I:\Anaconda\road\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to 

Extracting MNIST_data\train-images-idx3-ubyte.gz


W0813 01:18:16.041333  1932 deprecation.py:323] From I:\Anaconda\road\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0813 01:18:16.045340  1932 deprecation.py:323] From I:\Anaconda\road\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0813 01:18:16.129400  1932 deprecation.py:323] From I:\Anaconda\road\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for up

Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


### 2 设定批次

In [3]:
batch_size = 50
n_batch = mnist.train.num_examples // batch_size  # 总共有多少个批次

# 定义训练集占位符
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, shape=[None, 784], name='x_input')   # 每一行表示一个样本，内容是784为的像素值
    y = tf.placeholder(tf.float32, shape=[None, 10],  name='y_input')   # 每一行表示一个label，采用杜热编码    


In [4]:
# # 显示图片
with tf.name_scope('input_reshape'):
    image_reshaped_input = tf.reshape(x, [-1, 28, 28, 1])  # 28*28 灰度图片
    tf.summary.image('input', image_reshaped_input, 10)

### 3 创建安神经网络（不含隐藏层）

In [5]:
# with tf.name_scope('layer'):
#     with tf.name_scope('weights1'):
#         w1 = tf.Variable(tf.random_normal([784, 100]) * 0.01, name='w1')
#         variable_summary(w1)
#     with tf.name_scope('biases1'):
#         b1 = tf.Variable(tf.random_normal([100]) * 0.01, name='b1')
#     with tf.name_scope('wx_plus_b_l1'):
#         wx_plus_b_l1 = tf.matmul(x, w1) + b1
#     with tf.name_scope('tanh'):
#         l1 = tf.nn.tanh(wx_plus_b_l1)
        
#     with tf.name_scope('weights2'):
#         w2 = tf.Variable(tf.random_normal([100,10]) * 0.01, name='w2')
#     with tf.name_scope('biases2'):
#         b2 = tf.Variable(tf.random_normal([10]) * 0.01, name='b2')
#         variable_summary(b2)
#     with tf.name_scope('wx_plus_b_l2'):
#         wx_plus_b_l2 = tf.matmul(l1, w2) + b2
#     with tf.name_scope('softmax'):
#         prediction = tf.nn.softmax(wx_plus_b_l2)
        
        
with tf.name_scope('layer'):        
    w1 = tf.Variable(tf.truncated_normal([784, 2000], stddev=0.1))
    variable_summary(w1)
    b1 = tf.Variable(tf.zeros([2000]) + 0.1)
    wx_plus_b_l1 = tf.matmul(x, w1) + b1
    l1 = tf.nn.tanh(wx_plus_b_l1)
    l1_dropout = tf.nn.dropout(l1, 1)

    w2 = tf.Variable(tf.truncated_normal([2000, 1500], stddev=0.1))
    b2 = tf.Variable(tf.zeros([1500]) + 0.1)
    variable_summary(b2)
    wx_plus_b_l2 = tf.matmul(l1_dropout, w2) + b2
    l2 = tf.nn.tanh(wx_plus_b_l2)
    l2_dropout = tf.nn.dropout(l2, 1)

    w3 = tf.Variable(tf.truncated_normal([1500, 1000], stddev=0.1))
    b3 = tf.Variable(tf.zeros([1000]) + 0.1)
    wx_plus_b_l3 = tf.matmul(l2_dropout, w3) + b3
    l3 = tf.nn.tanh(wx_plus_b_l3)
    l3_dropout = tf.nn.dropout(l3, 1)

    w4 = tf.Variable(tf.truncated_normal([1000, 10], stddev=0.1))
    b4 = tf.Variable(tf.zeros([10]) + 0.1)
    wx_plus_b_l4 = tf.matmul(l3_dropout, w4) + b4
    l4 = tf.nn.tanh(wx_plus_b_l4)
    prediction = tf.nn.dropout(l4, 1)        

W0813 01:18:16.733828  1932 deprecation.py:506] From <ipython-input-5-4d71c2b81b9d>:29: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### 4 优化器

In [6]:
# loss = tf.reduce_mean(tf.square(y-prediction))
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
    tf.summary.scalar('loss', loss)
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.3).minimize(loss)

W0813 01:18:16.821888  1932 deprecation.py:323] From <ipython-input-6-32e2c0213e3b>:3: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### 5 获取预测结果并计算准确率

In [7]:
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'): 
        correct_prediction = tf.equal(tf.arg_max(prediction, 1), tf.arg_max(y, 1))
    with tf.name_scope('accuracy'): 
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar('accuracy', accuracy)

W0813 01:18:16.965011  1932 deprecation.py:323] From <ipython-input-7-d7023bafced3>:3: arg_max (from tensorflow.python.ops.gen_math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.math.argmax` instead


In [8]:
# 产生metadata文件  从测试集中选出3000个数字，并把他们的label保存到tsv文件中
# if tf.gfile.Exists(DIR + 'projector/projector/metedata.tsv'):
#     tf.gfile.DeleteRecursively(DIR + 'projector/projector/metedata.tsv')
with open (file=DIR + 'projector/projector/metadata.tsv', mode='w') as f:
    labels = sess.run(tf.arg_max(mnist.test.labels[:], 1))
    for i in range(image_num):
        f.write(str(labels[i]) + '\n')

In [9]:
merged = tf.summary.merge_all()  # 融合所有的summary

In [10]:
projector_writer = tf.summary.FileWriter(DIR + 'projector/projector/', sess.graph)
saver = tf.train.Saver()
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = embedding.name
embed.metadata_path = DIR + 'projector/projector/metadata.tsv'
embed.sprite.image_path = DIR + 'projector/data/mnist_10k_sprite.png'
embed.sprite.single_image_dim.extend([28,28])
projector.visualize_embeddings(projector_writer, config)                  
                                         

### 6 启动会话

In [11]:
sess.run(tf.global_variables_initializer())
for i in range(max_step):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    summary, _ = sess.run([merged, train_step], feed_dict={x:batch_x, y:batch_y}, options=run_options, run_metadata=run_metadata)
    projector_writer.add_run_metadata(run_metadata, 'step%03d'%i)
    projector_writer.add_summary(summary,i)
    
    if i%100==0:   
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})  # 用测试集来预测
        print("第{0}次  测试集准确率是： {1}".format(i, acc))
        
# 此处补上模型保存代码
saver.save(sess,DIR + 'projector/projector/a_model.ckpt', global_step=max_step)
projector_writer.close()
sess.close()

第0次  测试集准确率是： 0.29510000348091125
第100次  测试集准确率是： 0.8353999853134155
第200次  测试集准确率是： 0.8985000252723694
第300次  测试集准确率是： 0.9208999872207642
第400次  测试集准确率是： 0.9143999814987183
第500次  测试集准确率是： 0.9304999709129333
第600次  测试集准确率是： 0.9283999800682068
第700次  测试集准确率是： 0.9370999932289124
第800次  测试集准确率是： 0.9368000030517578
第900次  测试集准确率是： 0.9269999861717224
第1000次  测试集准确率是： 0.9383999705314636
